# Streaming estructurado con Azure Databricks en Power BI y Cosmos DB para el despliegue y análisis de tweets 

####Fuente:

Structured Streaming with Azure Databricks into Power BI & Cosmos DB

https://github.com/giulianorapoz/DatabricksStreamingPowerBI

In [1]:
%load_ext watermark
%watermark -a "Christian Castro" -u -d -p numpy,pandas,matplotlib
%watermark -a "Propiedad de DataIntelligence"

Christian Castro 
last updated: 2020-04-24 

numpy 1.18.1
pandas 1.0.1
matplotlib 3.1.3
Propiedad de DataIntelligence


En este notebook nos basaremos en el concepto de Streaming estructurado con Databricks y en el cómo se puede conectar directamente para utilizarlo junto con Power BI y Cosmos DB, lo que permite la visualización y análisis avanzados para una mayor disección de los datos consumidos por streaming estructurados. Construiremos una ruta de consumo de datos directamente con Azure Databricks que nos permitirá transmitir datos a un clúster de Apache Spark en tiempo casi real. Mostraremos algunas de las capacidades de análisis a las que se puede llamar directamente desde Databricks utilizando la API de Text Analytics, luego conectaremos Databricks directamente a Power BI para análisis e informes de disección de datos adicionales. Como paso final, leeremos y escribiremos desde Databricks directamente en CosmosDB como almacenamiento persistente y uso posterior.

El primer paso es configurar todos nuestros recursos individuales. Necesitaremos lo siguiente:

*    Un espacio de trabajo de Databricks y Apache spark cluster t. (Para ejecutar nuestros nuestros cuadernos).
*    Un centro de eventos, f. (Para que Databricks envíe los datos).
*    Una cuenta de servicios cognitivos t. (Para acceder a la API de Text Analytics).
*    Una aplicación de Twitter para los datos. (Para proporcionar la transmisión actual de datos).
*    Una base de datos CosmosDB como. (Para almacenar datos de forma persistente. Los datos)
*    Power BI Desktop para visualización de datos t. (Para visualizar y analizar los datos). 
     
Event Hub En primer lugar, cree un Event Hub buscando Event Hubs en Azure. En Crear espacio de nombres, elija un espacio de nombres y seleccione una suscripción de Azure, un grupo de recursos y una ubicación para crear el recurso.

In [ ]:
Corea central

nuevogrupo1

Haga clic en **Directivas de acceso compartido** y luego en RootManagedSharedAccessKey. Tome nota de la cadena de conexión y la clave principal ... las necesitará más adelante para permitir que Databricks envíe datos de permisos al Centro de eventos.

Clave principal
rxrkCqGNf2RQxgyAjEdrG0kJejAX0naaaik5eCVg+LU=

Clave secundaria
NTLRPa5dWrm1qOlKquvTiBPa9ChpXltlDur86suCNyA=

Cadena de conexión: clave principal
Endpoint=sb://tweetstopowerbi5.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=rxrkCqGNf2RQxgyAjEdrG0kJejAX0naaaik5eCVg+LU=
    
Cadena de conexión: clave secundaria   
Endpoint=sb://tweetstopowerbi5.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=NTLRPa5dWrm1qOlKquvTiBPa9ChpXltlDur86suCNyA=



A continuación, cree un Event Hub a través del espacio de nombres que acaba de crear. Haga clic en Event Hubs y luego + Agregar Event Hubs. Dale un nombre y luego presiona Crear.

Espacio de nombres: tweetstopowerbi4
Event hub: tweetstopowerbi4
    

¡Eso es todo, el centro de eventos **Event Hub** está listo para funcionar, y tenemos todas nuestras cadenas de conexión necesarias para que Databricks envíe datos!

Ahora construyamos un Databricks Workspace para crear el espacio de trabajo Databricks. Busque Databricks con Azure Portal. Proporcione un **nombre de área de trabajo**, su grupo de recursos y ubicación existentes y Premium como el nivel de precios más alto - (¡Nota importante !: para cConnection a través de DirectQuery a PowerBIPower BI no funcionará sin esto, ¡necesitará esto!)

In [ ]:
Recurso: tweetstopowerbi5
Grupo de recursos: nuevogrupo1

## Crear un clúster de Apache Spark dentro de Databricks

Para ejecutar notebooks para consumir el streming de data, primero se requiere un clúster. Para crear un clúster Apache Spark dentro de Databricks, cargue un Workspace desde el recurso Databricks se creó. Desde el portal de Databricks, seleccione Cluster.

dá click en: 'iniciar area de trabajo'

En un nuevo clúster, proporcione los siguientes valores para crear el clúster. ¡NOTA! - Para las capacidades de lectura / escritura en CosmosDB, se requiere una versión Apache Spark de 2.2.0. Al momento de escribir 2.3.0 aún no es compatible.

Adjuntar bibliotecas a Spark Cluster

Para permitir que la API de Twitter envíe tweets a Databricks y Databricks para leer y escribir datos en Event Hubs y CosmosDB, se requieren tres paquetes: 

• Conector Spark Event Hubs: com.microsoft.azure:azure-eventhubs-spark_2.11:2.3. 1 

• API de Twitter - org.twitter4j: twitter4j-core: 4.0.6 

• CosmosDB Spark Connector: http://repo1.maven.org/maven2/com/microsoft/azure/azure-cosmosdb-spark_2.2.0_2.11/ 1.1.1 / azure-cosmosdb-spark_2.2.0_2.11-1.1.1-uber.jar

Haga clic derecho en el espacio de trabajo de Databricks y seleccione Crear> Biblioteca. En la página Nueva biblioteca, seleccione Maven Coordinate e ingrese los nombres de las bibliotecas anteriores. se mantiene se encuentra aquí: https://github.com/Azure/azure-cosmosdb-spark

## Kit de herramientas de servicios cognitivos

Para calcular el sentimiento de los tweets, se requiere acceso a los Servicios Cognitivos de Microsoft. Esto permitirá que Databricks llame a la API de Text Analytics en tiempo casi real directamente desde el cuaderno y calcule el sentimiento de un tweet dado. Busque la API de Text Analytics en Azure Portal. Proporcione un nombre, ubicación y nivel de precios. (F0 será suficiente para los propósitos de esta demostración)

Una vez creado, haga clic en **Claves y puntos de conexión** y tome nota de la URL del punto final y la Clave principal que se utilizará. Estos valores serán necesarios para que Databricks llame con éxito a la API de Text Analytics.

endpoint:
https://tweetstopowerbi5.cognitiveservices.azure.com/


clave 1:

dae1210d95694ace8758d4cc0bea86d2

clave 2:

35a59dc1f1274710aa101ac432fb90d6


Crear notebooks Databricks

Para ejecutar el código, necesitaremos crear 4 cuadernos en el espacio de trabajo de Databricks creado de la siguiente manera:

* EventHubTweets (para enviar tweets al centro de eventos)
* TweetSentiment (Para calcular el sentimiento de la secuencia de tweets del centro de eventos)
* ScheduledDatasetCreation (Para crear y actualizar continuamente el conjunto de datos)
* DatasetValidation (para validar el conjunto de datos directamente dentro de Databricks)
